# FISH - Pipeline - A Python notebook to visualize FISH data

```
Author: Luis U. Aguilera
Contact Info: luis.aguilera@colostate.edu

Copyright (c) 2021 Munsky Group 
Colorado State University 
Licensed under BSD 3-Clause License.
```

### Libraries

In [ ]:
# Importing libraries
import sys
import pathlib
import warnings
import matplotlib.pyplot as plt
import pandas as pd
import glob
import numpy as np
import math
warnings.filterwarnings("ignore")

### Path to source directories

In [ ]:
# Defining directories
current_dir = pathlib.Path().absolute()
fa_dir = current_dir.parents[0].joinpath('src')

# Importing fish_analyses module
sys.path.append(str(fa_dir))
import fish_analyses as fa

# Path to credentials
desktop_path = pathlib.Path.home()/'Desktop'
path_to_config_file = desktop_path.joinpath('config.yml')


----

# Parameters defined by the user 

----

### Paths to images

In [ ]:
time_point = 'wo' # 'wo' '18min', '5h'
mandatory_substring='nuc_70__cyto_0__psfz_350__psfyx_160__ts_550_400'
connect_to_NAS = True

if time_point == 'wo':
    data_folder_path = pathlib.Path('smFISH_images/Linda_smFISH_images/Confocal/20220714/MS2-CY5_Cyto543_560_woStim')
    path_to_masks_dir = pathlib.Path('smFISH_images/Linda_smFISH_images/Confocal/20220714/MS2-CY5_Cyto543_560_woStim/masks_MS2-CY5_Cyto543_560_woStim___nuc_70__cyto_0.zip') 
    
if time_point == '18min':
    data_folder_path = pathlib.Path('smFISH_images/Linda_smFISH_images/Confocal/20220714/MS2-CY5_Cyto543_560_18minTPL_5uM')
    path_to_masks_dir = pathlib.Path('smFISH_images/Linda_smFISH_images/Confocal/20220714/MS2-CY5_Cyto543_560_18minTPL_5uM/masks_MS2-CY5_Cyto543_560_18minTPL_5uM___nuc_70__cyto_0.zip') 
    
if time_point == '5h':
    data_folder_path = pathlib.Path('smFISH_images/Linda_smFISH_images/Confocal/20220714/MS2-CY5_Cyto543_560_5hTPL_5uM')
    path_to_masks_dir = pathlib.Path('smFISH_images/Linda_smFISH_images/Confocal/20220714/MS2-CY5_Cyto543_560_5hTPL_5uM/masks_MS2-CY5_Cyto543_560_5hTPL_5uM___nuc_70__cyto_0.zip') 


In [ ]:
%%capture
list_images, list_masks, dataframe, number_images, number_color_channels = fa.Utilities.extract_images_masks_dataframe( data_folder_path,path_to_masks_dir, mandatory_substring, path_to_config_file,connect_to_NAS)

In [ ]:
# Select the cell to visualize
cell_id = 6

# Extracting image with cell and specific dataframe
subsection_image_with_selected_cell, df_spots = fa.Utilities.image_cell_selection(cell_id=cell_id, list_images=list_images, dataframe=dataframe)

In [ ]:
# Plot maximum projection
_, axes = plt.subplots(nrows = 1, ncols = number_color_channels, figsize = (15, 5))
for i in range(0, number_color_channels):
    axes[i].imshow( np.max(subsection_image_with_selected_cell[:,: ,:,i],axis=0))
    axes[i].grid(False)
    axes[i].set_xticks([])
    axes[i].set_yticks([])
    axes[i].set_title('max_z_channel_'+str(i) )

In [ ]:
# Plot all z_slices
number_z_slices = subsection_image_with_selected_cell.shape[0]
_, axes = plt.subplots(nrows = number_color_channels , ncols = number_z_slices, figsize = ( number_z_slices*2, 10 ))
for i in range(0, number_z_slices):
    for j in range(0, number_color_channels):
        axes[j,i].imshow( subsection_image_with_selected_cell[i,: ,:,j],cmap='Spectral')
        axes[j,i].grid(False)
        axes[j,i].set_xticks([])
        axes[j,i].set_yticks([])
        if i ==0:
            axes[j,i].set_ylabel('Channel_'+str(j) )
            
        if j == 0:
            axes[j,i].set_title('z_slice'+str(i) )

In [ ]:
df_spots